# Job Recommendations 

This notebook creates a model, to recommend job positions given a position requirements description . This is done only for IT jobs. 

In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score

    
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.



ModuleNotFoundError: No module named 'nltk'

In [8]:
data = pd.read_csv('data job posts.csv')
print(data.columns)
# selecting only IT Jobs
df = data[data['IT']]
# selecting 
cols = ['RequiredQual', 'Eligibility', 'Title', 'JobDescription', 'JobRequirment']
df=df[cols]
df.head(5)

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')


,RequiredQual,Eligibility,Title,JobDescription,JobRequirment
4,- University degree; economical background is ...,NaN,Software Developer,NaN,- Rendering technical assistance to Database M...
15,"- Excellent knowledge of Windows 2000 Server, ...",NaN,Network Administrator,NaN,- Network monitoring and administration;\r\n- ...
19,"As a GD you are creative, innovative and have\...",NaN,Graphic Designer,The position of Graphic Designer (GD) demands ...,Graphic Designer will be responsible for every...
34,Participants should be mid-level professionals...,NaN,Demographic Analysis Workshop,NaN,NaN
35,- Work experience of at least two years; \r\n-...,NaN,Programmer,NaN,NaN


# Modifying Job Titles
Selecting only top 21 job titles, to manage class imbalance

In [3]:
classes = df['Title'].value_counts()[:21]
keys = classes.keys().to_list()

df = df[df['Title'].isin(keys)]
df['Title'].value_counts()

Software Developer           134
Web Developer                101
Java Developer                88
Graphic Designer              75
Senior Java Developer         69
Software Engineer             69
PHP Developer                 65
Senior Software Engineer      63
Programmer                    56
IT Specialist                 55
Senior QA Engineer            43
Senior Software Developer     41
Android Developer             37
.NET Developer                36
Senior PHP Developer          34
Senior .NET Developer         34
iOS Developer                 31
Database Developer            29
Senior Web Developer          29
Software QA Engineer          29
Database Administrator        28
Name: Title, dtype: int64

Change job titles to base title. For exmaple, chaning Senior Java Developer to Java Developer.   

In [4]:
def chane_titles(x):
    x = x.strip()
    if x == 'Senior Java Developer':
        return 'Java Developer'
    elif x == 'Senior Software Engineer':
        return 'Software Engineer'
    elif x == 'Senior QA Engineer':
        return 'Software QA Engineer'
    elif x == 'Senior Software Developer':
        return 'Senior Web Developer'
    elif x =='Senior PHP Developer':
        return 'PHP Developer'
    elif x == 'Senior .NET Developer':
        return '.NET Developer'
    elif x == 'Senior Web Developer':
        return 'Web Developer'
    elif x == 'Database Administrator':
        return 'Database Admin/Dev'
    elif x == 'Database Developer':
        return 'Database Admin/Dev'

    else:
        return x
        
    
df['Title'] = df['Title'].apply(chane_titles)
df['Title'].value_counts()

Java Developer          157
Software Developer      134
Software Engineer       132
Web Developer           130
PHP Developer            99
Graphic Designer         75
Software QA Engineer     72
.NET Developer           70
Database Admin/Dev       57
Programmer               56
IT Specialist            55
Senior Web Developer     41
Android Developer        37
iOS Developer            31
Name: Title, dtype: int64

# Building custom tokenizer to process text

In [5]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        # lemmatize text - convert to base form 
        self.wnl = WordNetLemmatizer()
        # creating stopwords list, to ignore lemmatizing stopwords 
        self.stopwords = stopwords.words('english')
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.stopwords]

# removing new line characters, and certain hypen patterns                  
df['RequiredQual']=df['RequiredQual'].apply(lambda x: x.replace('\n', ' ').replace('\r', '').replace('- ', ''). replace(' - ', ' to '))

# Featurizing Text

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# train features and labels 
y = df['Title']
X = df['RequiredQual']
# tdif feature rep 
vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english')
vectorizer.fit(X)
# transoforming text to tdif features
tfidf_matrix = vectorizer.transform(X)
# sparse matrix to dense matrix for training
X_tdif = tfidf_matrix.toarray()
# encoding text labels in categories 
enc = LabelEncoder() 
enc.fit(y.values)
y_enc=enc.transform(y.values)

X_train_words, X_test_words, y_train, y_test = train_test_split(X, y_enc, test_size=0.15, random_state=10)

X_train = vectorizer.transform(X_train_words)
X_train = X_train.toarray()

X_test = vectorizer.transform(X_test_words)
X_test = X_test.toarray()


/opt/conda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# Training Naive Bayes
Looks pretty overfit

In [7]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
gnb = GaussianNB()
train_preds = gnb.fit(X_train, y_train).predict(X_train)
test_preds = gnb.predict(X_test)

print('Train acc: {0}'.format(accuracy_score(y_train, train_preds)))
print('Test acc: {0}'.format(accuracy_score(y_test, test_preds)))


Train acc: 0.9455852156057495
Test acc: 0.6918604651162791


# Training Logistic Regression
By modifiying the maximum number of iterations, and regularization, C, the above experienced overfitting was reduced significantly 


In [8]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(max_iter=15,verbose=1, C=0.75)

train_preds = logistic.fit(X_train, y_train).predict(X_train)
test_preds = logistic.predict(X_test)

print('Train acc: {0}'.format(accuracy_score(y_train, train_preds)))
print('Test acc: {0}'.format(accuracy_score(y_test, test_preds)))


[LibLinear]Train acc: 0.8757700205338809
Test acc: 0.7383720930232558


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Creating Job Recommendations 
Recommends 2 job position alternatives given a job requirement. By obtaining probability of class predictions, and picking the top N predictions, other than true label, N closest recommendations can be got

In [9]:
preds_data = {'Current Position Requirments': [], 'Current Position': [], 'Alternative 1': [], 'Alternative 2': []}
y_preds_proba = logistic.predict_proba(X_test)

counter = 0 
for idx, (pred_row, true_job_position) in enumerate(zip(y_preds_proba, y_test)):
    class_preds = np.argsort(pred_row)
    # delete true class
    for i in [-1, -2]:
        if class_preds[i] == true_job_position:
            class_preds=np.delete(class_preds,i)
    # getting other 2 highest job predictions         
    top_classes = class_preds[-2:]
    # obtaining class name string from int label 
    class_names = enc.inverse_transform(top_classes)
    true_job_position_name = enc.inverse_transform([true_job_position])
    # saving to dict
    preds_data['Current Position Requirments'].append(X_test_words.iloc[idx])
    preds_data['Current Position'].append(true_job_position_name[0])
    preds_data['Alternative 1'].append(class_names[1])
    preds_data['Alternative 2'].append(class_names[0])

    
    counter +=1

In [10]:
preds_df = pd.DataFrame.from_dict(preds_data)
preds_df.to_csv('Recommendations.csv', index=False)
preds_df


,Current Position Requirments,Current Position,Alternative 1,Alternative 2
0,From 1 to 5 years of experience in Android dev...,Android Developer,iOS Developer,Software Engineer
1,University degree in the appropriate field of ...,PHP Developer,Web Developer,Software Developer
2,At least 3 years of experience with PHP and Ja...,Web Developer,Software Engineer,PHP Developer
3,At least 3 years of work experience as a Java ...,Java Developer,Software Engineer,Software Developer
4,Bachelor??s degree in Engineering or Informati...,Software Engineer,Software Developer,Web Developer
...,...,...,...,...
167,"Artistic ability to draw, sketch and design cr...",Graphic Designer,Web Developer,Software Developer
168,"BS in Computer Science, Maths or a related dis...",Software Engineer,PHP Developer,Java Developer
169,University degree in Information Technology or...,IT Specialist,Software Developer,Database Admin/Dev
170,"University degree in Computer Science, Informa...",Software Developer,Java Developer,Software Engineer
